In [1]:
#Measure pitch of all wav files in directory
import glob
import numpy as np
import pandas as pd
import parselmouth

from parselmouth.praat import call
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


In [2]:
# This is the function to measure voice pitch
def measurePitch(voiceID, f0min, f0max, unit):
    sound = parselmouth.Sound(voiceID) # read the sound
    pitch = call(sound, "To Pitch", 0.0, f0min, f0max) #create a praat pitch object
    meanF0 = call(pitch, "Get mean", 0, 0, unit) # get mean pitch
    stdevF0 = call(pitch, "Get standard deviation", 0 ,0, unit) # get standard deviation
    harmonicity = call(sound, "To Harmonicity (cc)", 0.01, 75, 0.1, 1.0)
    hnr = call(harmonicity, "Get mean", 0, 0)
    pointProcess = call(sound, "To PointProcess (periodic, cc)", f0min, f0max)
    localJitter = call(pointProcess, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3)
    localabsoluteJitter = call(pointProcess, "Get jitter (local, absolute)", 0, 0, 0.0001, 0.02, 1.3)
    rapJitter = call(pointProcess, "Get jitter (rap)", 0, 0, 0.0001, 0.02, 1.3)
    ppq5Jitter = call(pointProcess, "Get jitter (ppq5)", 0, 0, 0.0001, 0.02, 1.3)
    ddpJitter = call(pointProcess, "Get jitter (ddp)", 0, 0, 0.0001, 0.02, 1.3)
    localShimmer =  call([sound, pointProcess], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    localdbShimmer = call([sound, pointProcess], "Get shimmer (local_dB)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    apq3Shimmer = call([sound, pointProcess], "Get shimmer (apq3)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    aqpq5Shimmer = call([sound, pointProcess], "Get shimmer (apq5)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    apq11Shimmer =  call([sound, pointProcess], "Get shimmer (apq11)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    ddaShimmer = call([sound, pointProcess], "Get shimmer (dda)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    

    return meanF0, stdevF0, hnr, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer

In [3]:
def runPCA(df):
    #Z-score the Jitter and Shimmer measurements
    features = ['localJitter', 'localabsoluteJitter', 'rapJitter', 'ppq5Jitter', 'ddpJitter',
                'localShimmer', 'localdbShimmer', 'apq3Shimmer', 'apq5Shimmer', 'apq11Shimmer', 'ddaShimmer']
    # Separating out the features
    x = df.loc[:, features].values
    # Separating out the target
    #y = df.loc[:,['target']].values
    # Standardizing the features
    x = StandardScaler().fit_transform(x)
    #PCA
    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(x)
    principalDf = pd.DataFrame(data = principalComponents, columns = ['JitterPCA', 'ShimmerPCA'])
    principalDf
    return principalDf

In [5]:
# create lists to put the results
file_list = []
mean_F0_list = []
sd_F0_list = []
hnr_list = []
localJitter_list = []
localabsoluteJitter_list = []
rapJitter_list = []
ppq5Jitter_list = []
ddpJitter_list = []
localShimmer_list = []
localdbShimmer_list = []
apq3Shimmer_list = []
aqpq5Shimmer_list = []
apq11Shimmer_list = []
ddaShimmer_list = []

# Go through all the wave files in the folder and measure pitch
for wave_file in glob.glob("audio/*.wav"):
    sound = parselmouth.Sound(wave_file)
    (meanF0, stdevF0, hnr, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer) = measurePitch(sound, 75, 500, "Hertz")
    file_list.append(wave_file) # make an ID list
    mean_F0_list.append(meanF0) # make a mean F0 list
    sd_F0_list.append(stdevF0) # make a sd F0 list
    hnr_list.append(hnr)
    localJitter_list.append(localJitter)
    localabsoluteJitter_list.append(localabsoluteJitter)
    rapJitter_list.append(rapJitter)
    ppq5Jitter_list.append(ppq5Jitter)
    ddpJitter_list.append(ddpJitter)
    localShimmer_list.append(localShimmer)
    localdbShimmer_list.append(localdbShimmer)
    apq3Shimmer_list.append(apq3Shimmer)
    aqpq5Shimmer_list.append(aqpq5Shimmer)
    apq11Shimmer_list.append(apq11Shimmer)
    ddaShimmer_list.append(ddaShimmer)
df = pd.DataFrame(np.column_stack([file_list, mean_F0_list, sd_F0_list, hnr_list, localJitter_list, localabsoluteJitter_list, rapJitter_list, ppq5Jitter_list, ddpJitter_list, localShimmer_list, localdbShimmer_list, apq3Shimmer_list, aqpq5Shimmer_list, apq11Shimmer_list, ddaShimmer_list]),
                               columns=['voiceID', 'meanF0Hz', 'stdevF0Hz', 'HNR', 'localJitter', 'localabsoluteJitter', 'rapJitter', 
                                        'ppq5Jitter', 'ddpJitter', 'localShimmer', 'localdbShimmer', 'apq3Shimmer', 'apq5Shimmer', 
                                        'apq11Shimmer', 'ddaShimmer'])  #add these lists to pandas in the right order
pcaData = runPCA(df)

df = pd.concat([df, pcaData], axis=1)

# Write out the updated dataframe
df.to_csv("processed_results.csv", index=False)

/home/david/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [6]:
df

,voiceID,meanF0Hz,stdevF0Hz,HNR,localJitter,localabsoluteJitter,rapJitter,ppq5Jitter,ddpJitter,localShimmer,localdbShimmer,apq3Shimmer,apq5Shimmer,apq11Shimmer,ddaShimmer,JitterPCA,ShimmerPCA
0,audio/m1753_hi.wav,87.16159195844318,11.22980741163208,13.42652993981289,0.02731935503397948,0.00032418850064155994,0.010041859585134545,0.006260211530670556,0.030125578755403636,0.13699953404927798,1.4311422139546028,0.04166371793276076,0.07019764974155855,0.1349010811549323,0.12499115379828228,0.649802,2.309385
1,audio/m1759_hi.wav,115.80369287668644,13.883171218679545,11.392696491083685,0.014911312339692274,0.0001287318774037279,0.004444796006119185,0.004115627546060311,0.013334388018357554,0.1046929160170572,1.0432753697321855,0.04377824995208884,0.055826821602754456,0.0925686886176464,0.1313347498562665,0.201645,-2.768259
2,audio/m1770_hi.wav,108.91294049116742,17.733449636661522,6.757093382626961,0.02423711339423511,0.00022328428209636035,0.007378013292925969,0.009604795098024465,0.022134039878777906,0.09636612501823205,0.9356901088484686,0.03036856536706759,0.050809744460769496,0.12893557383889281,0.09110569610120277,-1.901345,0.274877
3,audio/m1762_hi.wav,139.73791943140907,15.666261055367125,8.086574383935542,0.022329427277179698,0.00016063190341751164,0.00539978268052617,0.00716663901222205,0.01619934804157851,0.08406045507059069,1.1027738240526483,0.033230353099084876,0.04832951007746663,0.10057969049670032,0.09969105929725464,-1.360620,-1.296819
4,audio/m1765_hi.wav,103.32997496404903,9.72712320127043,12.618495024640444,0.01883779990319546,0.0001827953567167917,0.008690646386717427,0.007937250581257547,0.026071939160152283,0.09390517452240592,1.0464113087741906,0.033155004867667946,0.046343832568742256,0.12036992250592413,0.09946501460300383,-1.411588,-0.279005
5,audio/m1758_hi.wav,109.81550992689377,9.799585845065458,6.39876198583097,0.02973322150319197,0.0002718531631692118,0.013058115272767642,0.008617669567038846,0.039174345818302925,0.12531616558781422,1.1603757187159636,0.053841878850570984,0.09214012287143829,0.11545879993517703,0.16152563655171295,0.582257,3.444366
6,audio/m1776_hi.wav,83.85498236083922,3.9696716154920897,4.667742994612689,0.017634920572426056,0.00022676098090127335,0.00657629001395815,0.008381065565158738,0.01972887004187445,0.10347309003663958,0.8474130157052167,0.03282691784246129,0.06014227067818171,0.1835606891256708,0.09848075352738388,-0.941891,-0.626310
7,audio/m1750_hi.wav,117.94152882236595,19.257673917415236,7.395675800674666,0.020778622456621224,0.00017821329031250202,0.002732818644849007,0.005681742599396066,0.00819845593454702,0.1490436892314192,1.2043520515481125,0.08450036151143482,0.08311163813651551,0.11031822440022865,0.2535010845343045,3.463236,-1.494111
8,audio/m1766_hi.wav,112.76910497428926,6.462185004171308,10.448668639858692,0.021694823663519054,0.00019295752590071347,0.011433524513060206,0.007456777780452748,0.03430057353918062,0.17846151383116235,1.7117967534365877,0.07482011594677787,0.10684494043422528,0.2143689459296592,0.2244603478403336,4.641256,2.319600
9,audio/m1780_hi.wav,154.06357379371164,113.41760172240889,7.851113935311286,0.027943225653585792,0.0001966943844835363,0.007545270398162258,0.008852296748014684,0.022635811194486773,0.08228355105475352,0.6611848677730496,0.03908726783154418,0.03839054687574852,0.0718911518492142,0.11726180349463255,-2.613519,0.018476
